In [1]:
import pathlib
import sys
PATH = pathlib.Path().absolute().resolve()

# %matplotlib inline
import math
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tk
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.4)
COLORS = sns.color_palette()

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
def ax_style(ax):
    ax.grid(which='minor', axis='x', dashes=(5, 5), linewidth=.5)
    ax.grid(which='minor', axis='y', dashes=(5, 5), linewidth=.5)
    ax.legend()
    ax.minorticks_on()
    return ax


In [25]:
@interact(
    a=widgets.FloatText(value=10, description="Maximum y"),
    b=widgets.FloatText(value=8, description="Stop x"),
    c=widgets.FloatText(value=1, description="Distance penalty")
)
def _(a, b, c):
    functions = [
        (lambda d: (d + 1) ** 1.3, 'x'),
        (lambda d: (a - 1) * (1 - math.e ** -((d / b) ** 2)) + 1 + d / c, 'custom'),
    ]
    fig = plt.figure(figsize=(20, 10))

    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(222)
    ax3 = fig.add_subplot(223)
    ax4 = fig.add_subplot(224)
    X = np.linspace(0, 64, 100)
    for i, (fn, name) in enumerate(functions):
        v_fn = np.vectorize(fn)
        Y = v_fn(X)
        dY = np.zeros(Y.shape, np.float)
        dY[0:-1] = np.diff(Y) / np.diff(X)
        dY[-1] = (Y[-1] - Y[-2]) / (X[-1] - X[-2])
        ax1.plot(X, Y, label=name, color=COLORS[i % len(COLORS)])
        ax2.plot(X, dY, label=f"({name})`", color=COLORS[i % len(COLORS)])
        ax3.plot(X, 1 / Y, label=name, color=COLORS[i % len(COLORS)])
        ax4.plot(X, 1 / Y, label=name, color=COLORS[i % len(COLORS)])

    ax_style(ax1)
    ax_style(ax2)
    ax_style(ax3)
    ax_style(ax4)
    ax4.set_yscale("log")
    fig.tight_layout()
    plt.show()
    
    print(f"lambda d: {a - 1} * (1 - math.e ** -((d / {b}) ** 2)) + 1 + d / {c}")

interactive(children=(FloatText(value=10.0, description='Maximum y'), FloatText(value=8.0, description='Stop x…